In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
image = ee.Image('USGS/SRTMGL1_003').select('elevation')
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTMGL1')
Map


Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [4]:
roi = Map.user_roi
clipped_image = image.clip(roi)

Map.centerObject(roi)
Map


Map(bottom=3310.0, center=[43.004647127794435, 141.74560546875003], controls=(WidgetControl(options=['position…

In [5]:
features = geemap.fishnet(roi, rows=5, cols=5)
Map.addLayer(features, {}, 'Grids')

In [6]:
geemap.download_ee_image_tiles(
    image, features, '/Users/gigisung/gigi_git/undatathon/nk_nationtal/dem_national', prefix="dem_", crs="EPSG:5186", scale=3
)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the dem_01.tif download size (raw: 3.85 GB).


dem_01.tif: |          | 0.00/3.85G (raw) [  0.0%] in 00:00 (eta:     ?)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the dem_02.tif download size (raw: 3.72 GB).


dem_02.tif: |          | 0.00/3.72G (raw) [  0.0%] in 00:00 (eta:     ?)

## Merge

In [ ]:
import rasterio
from rasterio.merge import merge
import glob
import os

# Get a list of all the tile files
search_criteria = "/Users/gigisung/gigi_git/undatathon/nk_nationtal/dem/*.tif"
tile_files = glob.glob(search_criteria)

# Open the tile files and add them to a list
src_files_to_mosaic = [rasterio.open(fp) for fp in tile_files]

# Merge the tile files
mosaic, out_trans = merge(src_files_to_mosaic)

# Write the merged file to disk
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": src_files_to_mosaic[0].crs
                 }
                )

out_fp = "/Users/gigisung/gigi_git/undatathon/nk_nationtal/dem/nk_national_merged_dem.tif"
with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)


In [ ]:
# Define the coordinates for the polygon
coordinates = [
    [129.696351, 42.407487],
    [129.696351, 42.450693],
    [129.797631, 42.450693],
    [129.797631, 42.407487],
    [129.696351, 42.407487]
]

# Create the polygon object
polygon = ee.Geometry.Polygon(coordinates, proj='EPSG:4326', geodesic=False)
Map.addLayer(polygon, {}, 'hoerong')
Map


In [ ]:
clipped_image = image.clip(polygon)

clipped_image

In [ ]:
geemap.ee_export_image(clipped_image, filename='/Users/gigisung/gigi_git/undatathon/nk_nationtal/dem/huerong_dem_clipped.tif', scale=3)
